# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.86/12342/1 Dashboard: http://10.20.0.86:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1015,George,-0.43948506654452735,0.9514584179480028
2000-01-01 00:00:01,987,Alice,-0.673752290628373,0.03070770634464104
2000-01-01 00:00:02,1014,Oliver,0.12174010964448101,-0.5342745157595237
2000-01-01 00:00:03,1022,Norbert,0.153004375239856,-0.7493963621311339
2000-01-01 00:00:04,1036,Victor,-0.5663715710765913,0.08688017795896896
2000-01-01 00:00:05,947,Tim,0.1492600194162097,-0.10118615549429943
2000-01-01 00:00:06,971,Bob,0.5474609080372728,-0.20547912734886364
2000-01-01 00:00:07,993,Yvonne,0.2083578809390101,-0.7932654799433532
2000-01-01 00:00:08,1018,Victor,-0.6537458172119728,0.15776759999556167


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1024,Sarah,-0.9817825504702908,-0.6425638067756481
2000-01-30 00:00:01,946,Ursula,0.6325498340098568,0.4380340632742876
2000-01-30 00:00:02,1030,Victor,0.0720010971766083,-0.7385026824050394
2000-01-30 00:00:03,966,Yvonne,-0.033666805428369306,-0.7074376540666227
2000-01-30 00:00:04,954,Wendy,-0.7168049362402449,0.033336668522342405
2000-01-30 00:00:05,1013,Patricia,-0.6062789342779971,0.19381326221658446
2000-01-30 00:00:06,999,Ray,-0.0031913440149069494,0.8030584824955296
2000-01-30 00:00:07,988,Alice,0.9494061578240869,-0.17139837408245384
2000-01-30 00:00:08,997,Alice,0.06481660748405593,-0.03795687126764835


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1015,George,-0.439485,0.951458
1,2000-01-01 00:00:01,987,Alice,-0.673752,0.030708
2,2000-01-01 00:00:02,1014,Oliver,0.121740,-0.534275
3,2000-01-01 00:00:03,1022,Norbert,0.153004,-0.749396
4,2000-01-01 00:00:04,1036,Victor,-0.566372,0.086880


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1015,George,-0.439485,0.951458
1,2000-01-01 00:00:01,987,Alice,-0.673752,0.030708
2,2000-01-01 00:00:02,1014,Oliver,0.121740,-0.534275
3,2000-01-01 00:00:03,1022,Norbert,0.153004,-0.749396
4,2000-01-01 00:00:04,1036,Victor,-0.566372,0.086880


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.47 s, sys: 572 ms, total: 5.04 s
Wall time: 3.28 s


name
Alice      -0.000222
Bob        -0.000795
Charlie    -0.001745
Dan         0.000391
Edith       0.003001
Frank      -0.000266
George      0.000604
Hannah      0.000606
Ingrid     -0.001594
Jerry       0.000467
Kevin       0.001625
Laura       0.000761
Michael     0.000088
Norbert    -0.000402
Oliver     -0.002197
Patricia   -0.003467
Quinn      -0.003289
Ray        -0.000175
Sarah      -0.000239
Tim        -0.001845
Ursula     -0.001018
Victor      0.002333
Wendy       0.000875
Xavier      0.000679
Yvonne     -0.000101
Zelda      -0.003340
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.38 s, sys: 143 ms, total: 1.52 s
Wall time: 1.02 s


name
Alice      -0.000222
Bob        -0.000795
Charlie    -0.001745
Dan         0.000391
Edith       0.003001
Frank      -0.000266
George      0.000604
Hannah      0.000606
Ingrid     -0.001594
Jerry       0.000467
Kevin       0.001625
Laura       0.000761
Michael     0.000088
Norbert    -0.000402
Oliver     -0.002197
Patricia   -0.003467
Quinn      -0.003289
Ray        -0.000175
Sarah      -0.000239
Tim        -0.001845
Ursula     -0.001018
Victor      0.002333
Wendy       0.000875
Xavier      0.000679
Yvonne     -0.000101
Zelda      -0.003340
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.21 s, sys: 86.1 ms, total: 1.29 s
Wall time: 896 ms


name
Alice      -0.000222
Bob        -0.000795
Charlie    -0.001745
Dan         0.000391
Edith       0.003001
Frank      -0.000266
George      0.000604
Hannah      0.000606
Ingrid     -0.001594
Jerry       0.000467
Kevin       0.001625
Laura       0.000761
Michael     0.000088
Norbert    -0.000402
Oliver     -0.002197
Patricia   -0.003467
Quinn      -0.003289
Ray        -0.000175
Sarah      -0.000239
Tim        -0.001845
Ursula     -0.001018
Victor      0.002333
Wendy       0.000875
Xavier      0.000679
Yvonne     -0.000101
Zelda      -0.003340
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html